<a href="https://colab.research.google.com/github/khalidpark/deeplearning_whitepaper/blob/main/deep_learning_whitepaper_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# early stop

In [2]:
import numpy as np

# Tensorflow에서 데이터를 가져와 규제 하는 코드

from tensorflow.keras.datasets import fashion_mnist

# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
print(X_train.shape, X_test.shape)

# 데이터를 정규화 합니다
X_train = X_train / 255.
X_test = X_test /255.

# 클래스를 확인합니다.
np.unique(y_train)


(60000, 28, 28) (10000, 28, 28)


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [3]:
# 기본적인 신경망을 만드는 코드

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import keras, os

# 모델 구성을 확인합니다.
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(10, activation='softmax')
])
# 업데이트 방식을 설정합니다.
model.compile(optimizer='adam'
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
model.summary()
# 총 7850 parameters (10 bias)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [5]:
import tensorflow as tf

# 모델 학습을 위한 코드

# 변수 설정을 따로 하는 방법을 적용하기 위한 코드입니다. 
batch_size = 30
epochs_max = 1

# 학습시킨 데이터를 저장시키기 위한 코드입니다. 
checkpoint_filepath = "FMbest.hdf5"

# overfitting을 방지하기 위해서 학습 중 early stop을 수행하기 위한 코드입니다.
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

# Validation Set을 기준으로 가장 최적의 모델을 찾기 위한 코드입니다.
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

# 모델 학습 코드 + early stop + Best model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs_max, verbose=1, 
          validation_data=(X_test,y_test), 
          callbacks=[early_stop, save_best])


2000/2000 [==============================] - 5s 2ms/step - loss: 0.7681 - accuracy: 0.7413 - val_loss: 0.5077 - val_accuracy: 0.8248

Epoch 00001: val_loss improved from inf to 0.50770, saving model to FMbest.hdf5


In [6]:
# 학습된 모델을 이용하여 테스트하는 코드

model.predict(X_test[0:1])
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)


313/313 - 0s - loss: 0.5077 - accuracy: 0.8248


In [7]:
!ls


FMbest.hdf5  sample_data


In [8]:
# 체크포인트에 저장된 가중치들을 불러들이는 코드

model.load_weights(checkpoint_filepath)


In [9]:
# best model을 이용한 테스트 데이터 예측 정확도 재확인 코드

model.predict(X_test[0:1])
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)


313/313 - 0s - loss: 0.5077 - accuracy: 0.8248




---



# weight decay